In [18]:
import datasets
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List

In [19]:
train_data = datasets.load_dataset("wmt16","de-en", split="train[:50000]")

In [20]:
val_data = datasets.load_dataset("wmt16","de-en", split="validation")
test_data = datasets.load_dataset("wmt16","de-en", split="test")

In [21]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

50000
2169
2999


In [22]:
from torchtext.data.utils import get_tokenizer

tokenizer_german = get_tokenizer("spacy", language="de_core_news_sm")
tokenizer_english = get_tokenizer("spacy", language="en_core_web_sm")

In [23]:
# Set source and target languages
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

# Place-holders for the token transformers and vocabularies
token_transform = {SRC_LANGUAGE: tokenizer_german, TGT_LANGUAGE: tokenizer_english}
vocab_transform = {}

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

In [24]:
# Helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    for data_sample in data_iter:
        yield token_transform[language](data_sample['translation'][language])

# Build vocabularies for both languages
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Directly passing the dataset to yield_tokens without using .map
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_data, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)
    # Set UNK_IDX as the default index for vocabularies
    vocab_transform[ln].set_default_index(UNK_IDX)

In [27]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [28]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [31]:
from torch.nn.utils.rnn import pad_sequence

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz))) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        # Tokenize the source and target text samples
        src_tokens = tokenizer_german(src_sample)
        tgt_tokens = tokenizer_english(tgt_sample)
        
        # Numericalize tokens and append to respective batch lists
        src_batch.append(torch.tensor(vocab_transform[SRC_LANGUAGE](src_tokens), dtype=torch.long))
        tgt_batch.append(torch.tensor(vocab_transform[TGT_LANGUAGE](tgt_tokens), dtype=torch.long))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX, batch_first=False).to(DEVICE)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX, batch_first=False).to(DEVICE)

    # Create masks
    src_mask = generate_square_subsequent_mask(src_batch.size(0)).to(DEVICE)
    tgt_mask = generate_square_subsequent_mask(tgt_batch.size(0)).to(DEVICE)
    src_padding_mask = (src_batch == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt_batch == PAD_IDX).transpose(0, 1)
    memory_key_padding_mask = src_padding_mask.clone()

    return src_batch, tgt_batch, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, memory_key_padding_mask


from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]['translation'][SRC_LANGUAGE], self.data[idx]['translation'][TGT_LANGUAGE]

train_dataset = CustomDataset(train_data)
val_dataset = CustomDataset(val_data)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

In [34]:
from tqdm import tqdm

def train_epoch(model, optimizer, dataloader, loss_fn, device):
    model.train()
    losses = 0
    for src, tgt, src_mask, _, src_padding_mask, _, memory_key_padding_mask in tqdm(dataloader, desc="Training"):
        src, tgt = src.to(device), tgt.to(device)

        # Reduce the target sequence by 1 for the input to the decoder
        tgt_input = tgt[:-1, :]
        tgt_output = tgt[1:, :]  # This is used for loss calculation

        # Adjust the size of the target padding mask to match tgt_input
        tgt_padding_mask = (tgt_input == PAD_IDX).transpose(0, 1).to(device)
        
        # Regenerate tgt_mask to match the size of tgt_input
        tgt_mask = generate_square_subsequent_mask(tgt_input.size(0)).to(device)

        # Ensure src_mask and memory_key_padding_mask are correctly sized and on the correct device
        src_mask = src_mask.to(device)
        memory_key_padding_mask = src_padding_mask.clone().to(device)  # Assuming src_padding_mask needs no adjustment

        logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, memory_key_padding_mask)

        optimizer.zero_grad()

        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_output.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(dataloader)


def evaluate(model, dataloader, loss_fn, device):
    model.eval()
    losses = 0
    with torch.no_grad():
        for src, tgt, src_mask, _, src_padding_mask, tgt_padding_mask, memory_key_padding_mask in tqdm(dataloader, desc="Evaluating"):
            src, tgt = src.to(device), tgt.to(device)
            
            tgt_input = tgt[:-1, :]
            tgt_output = tgt[1:, :]
            
            # Regenerate tgt_mask because tgt_input has changed
            tgt_mask = generate_square_subsequent_mask(tgt_input.size(0)).to(device)
            
            src_mask, src_padding_mask, tgt_padding_mask, memory_key_padding_mask = src_mask.to(device), src_padding_mask.to(device), tgt_padding_mask.to(device), memory_key_padding_mask.to(device)
            
            logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
            
            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_output.reshape(-1))
            losses += loss.item()
    
    return losses / len(dataloader)


NUM_EPOCHS = 20

for epoch in range(1, NUM_EPOCHS+1):
    train_loss = train_epoch(transformer, optimizer, train_dataloader, loss_fn, DEVICE)
    val_loss = evaluate(transformer, val_dataloader, loss_fn, DEVICE)
    print(f"Epoch {epoch}, Train Loss: {train_loss}, Val Loss: {val_loss}")


Training:   1%|          | 2/391 [01:09<3:44:42, 34.66s/it]


KeyboardInterrupt: 